<a href="https://colab.research.google.com/github/bur3hani/mtaalam-training/blob/main/Another_copy_of_Untitled7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
from datasets import load_dataset

# Load Kiswahili corpus (OSCAR Dataset)
dataset = load_dataset("oscar", "unshuffled_deduplicated_sw")

# Print sample data
print(dataset["train"][1]["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/303k [00:00<?, ?B/s]

oscar.py:   0%|          | 0.00/14.8k [00:00<?, ?B/s]

The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/24803 [00:00<?, ? examples/s]

Miripuko hiyo inakuja mwanzoni mwa Wiki Takatifu kuelekea Pasaka na ikiwa ni wiki chache tu kabla ya Papa Francis kuanza ziara yake katika nchi hiyo yenye idadi kubwa kabisa ya watu katika ulimwengu wa nchi za Kiarabu.


In [ ]:
from transformers import AutoTokenizer

# Load mT5 tokenizer with fast processing
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", use_fast=True)

# Define max_length to avoid warning
MAX_LENGTH = 512  # Adjust as needed
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

# Apply tokenization to dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/24803 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments

# Load Kiswahili dataset
dataset = load_dataset("oscar", "unshuffled_deduplicated_sw")

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small", use_fast=True)

# Tokenize dataset
MAX_LENGTH = 512  # Ensures no truncation warnings

def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=MAX_LENGTH)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split dataset into train (90%) and validation (10%)
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
dataset = DatasetDict({
    "train": train_test_split["train"],
    "validation": train_test_split["test"]  # Rename test set as validation
})


In [ ]:
# Load mT5-small model
model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
)


In [ ]:
# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
)


In [ ]:
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        inputs = self.tokenizer(item["input"], return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        outputs = self.tokenizer(item["output"], return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        return {
            "input_ids": inputs.input_ids.flatten(),
            "attention_mask": inputs.attention_mask.flatten(),
            "labels": outputs.input_ids.flatten(), #labels are used as decoder_input_ids
        }

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

data = [{"input":"translate English to German: Hello, world!","output":"Hallo Welt!"}]

dataset = MyDataset(data, tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


TrainOutput(global_step=1, training_loss=66.59555053710938, metrics={'train_runtime': 80.5492, 'train_samples_per_second': 0.012, 'train_steps_per_second': 0.012, 'total_flos': 132187422720.0, 'train_loss': 66.59555053710938, 'epoch': 1.0})

In [ ]:
import torch
from transformers import MT5Tokenizer, MT5ForConditionalGeneration, Trainer, TrainingArguments
from torch.utils.data import Dataset
import wandb

class MyDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        inputs = self.tokenizer(item["input"], return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        outputs = self.tokenizer(item["output"], return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        return {
            "input_ids": inputs.input_ids.flatten(),
            "attention_mask": inputs.attention_mask.flatten(),
            "labels": outputs.input_ids.flatten(), #labels are used as decoder_input_ids
        }

# Initialize wandb
wandb.init(project="mt5-training")

tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-small")

data = [{"input":"translate English to German: Hello, world!","output":"Hallo Welt!"},{"input":"translate English to German: How are you?","output":"Wie geht es dir?"}]

dataset = MyDataset(data, tokenizer)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    logging_dir='./logs',
    logging_steps=10,
    report_to="wandb"  # Enable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

wandb.finish() # finish the wandb run.

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.


Step,Training Loss


train/epoch,▁▁
train/global_step,▁█
total_flos,264374845440.0
train/epoch,1
train/global_step,2
train_loss,67.18486
train_runtime,117.2928
train_samples_per_second,0.017
train_steps_per_second,0.017


In [ ]:
# Save trained model
model.save_pretrained("kiswahili_grammar_model")
tokenizer.save_pretrained("kiswahili_grammar_model")

('kiswahili_grammar_model/tokenizer_config.json',
 'kiswahili_grammar_model/special_tokens_map.json',
 'kiswahili_grammar_model/spiece.model',
 'kiswahili_grammar_model/added_tokens.json')